In [1]:
from __future__ import print_function
import torch.utils.data as data
from PIL import Image
import os
import sys
import pickle
import numpy as np
import lmdb
import torch
import torchvision.transforms as transforms
from torch import nn
import torch.optim as optim
import torch
from torch.autograd import Variable
import json
from Models import LSTM, GCN, RESNET
from Loader import Loader
import json



## train Method 
By getting generated training data, and the lstm and resnet and the model trained with GCD, we train our model

In [2]:
def train(X, lstm,model,learning_rate=0.01):
    hx, cx = lstm.initHidden()
    optimizer = optim.Adam(lstm.parameters(), learning_rate)
    criterion = nn.CosineEmbeddingLoss(0.1).cuda()
    
    for epoch in range(0, 10):
        loss=0
        for i, (img, instrs, ingrs, category, target) in enumerate(X):
         if i==1:
          break 
        
         for j in range(len(img)): #num_iters=6
            image_vector=model(torch.Tensor(img[j]).unsqueeze(0))  # Tensor (1,1024)f
            # print("image_vector",image_vector)
        

            tar = Variable(torch.FloatTensor(1))
            print(tar)
            tar.data[0] = target[j]

            for z in range(len(instrs[0])): #num_iters=20
                         
                # Iterating on lstm units
                lstm_input = instrs[0][z].unsqueeze(0).unsqueeze(0)
                lstm_output, hx, cx = lstm(lstm_input, hx, cx) ##hx Tensor (1,1,1024)
                                

            loss += criterion( hx, image_vector.unsqueeze(0),tar[0])
            loss.backward(retain_graph=True)
            optimizer.step()

         print("loss",loss.data.item()/20)
    

This method convert images in jpeg format to 1024 dimension vector using resnet

In [3]:
def image2vector(X,model):
    vector2image=dict()
    vector_images=[]

    for i, (img, instrs, ingrs, category, target) in enumerate(X):
        if i==1:
            break
        for j in range(len(img)): #num_iters=6
            image=torch.Tensor(img[j]).unsqueeze(0)
            vector = model(torch.Tensor(image))  # Tensor (1,1024)
            vector2image[vector]= image
            vector_images.append(vector)
    return vector_images,vector2image

This method sort all the images and ranked them based on cosine similarity to a given recipe

In [ ]:

def rank( imgs, instrs, ingrs):
    random.seed(opts.seed)
    type_embedding = opts.embtype 
    im_vecs = imgs 
    instr_vecs = instrs 
    names = rec_ids

    # Sort based on names to always pick same samples for medr
    idxs = np.argsort(names)
    names = names[idxs]

    # Ranker
    N = opts.medr
    idxs = range(N)

    glob_rank = []
    glob_recall = {1:0.0,5:0.0,10:0.0}
    for i in range(10):

        ids = random.sample(xrange(0,len(names)), N)
        im_sub = im_vecs[ids,:]
        instr_sub = instr_vecs[ids,:]
        ids_sub = names[ids]

        # if params.embedding == 'image':
        if type_embedding == 'image':
            sims = np.dot(im_sub,instr_sub.T) # for im2recipe
        else:
            sims = np.dot(instr_sub,im_sub.T) # for recipe2im

        med_rank = []
        recall = {1:0.0,5:0.0,10:0.0}

        for ii in idxs:

            name = ids_sub[ii]
            # get a column of similarities
            sim = sims[ii,:]

            # sort indices in descending order
            sorting = np.argsort(sim)[::-1].tolist()

            # find where the index of the pair sample ended up in the sorting
            pos = sorting.index(ii)

            if (pos+1) == 1:
                recall[1]+=1
            if (pos+1) <=5:
                recall[5]+=1
            if (pos+1)<=10:
                recall[10]+=1

            # store the position
            med_rank.append(pos+1)

        for i in recall.keys():
            recall[i]=recall[i]/N

        med = np.median(med_rank)
        # print "median", med

        for i in recall.keys():
            glob_recall[i]+=recall[i]
        glob_rank.append(med)

    for i in glob_recall.keys():
        glob_recall[i] = glob_recall[i]/10

    return np.average(glob_rank), glob_recall, i, sorting[0]


## test method
In the test method we compare all of the train images with a query recipe (from test dataset) in order to retrive the most k similar ones.

In [4]:
def test(X_train,X_test ,lstm,resnet, gcn, learning_rate=0.01):
    hx, cx = lstm.initHidden()

    vector_images, vector2image=image2vector(X_train,resnet)
    i=-1
    predicted_images=[]
    for i, (img, instrs, ingrs, category, target) in enumerate(X_test):
        if i==1:
          break 
        for j in range(len(img)): #num_iters=6
            y=resnet(torch.Tensor(img[j]).unsqueeze(0))
            
            for z in range(len(instrs[0])): #num_iters=20
                         
                # Iterating on lstm units
                lstm_input = instrs[0][z].unsqueeze(0).unsqueeze(0)
                lstm_output, hx, cx = lstm(lstm_input, hx, cx) ##hx Tens
            best_cos =torch.Tensor([-1])
            y_predict= None
            for v in vector_images: # resn
                  cos = nn.CosineSimilarity(dim=1, eps=1e-6)
                  output = cos(hx[0], v)

                  if output>best_cos:
                   best_cos=output
                   y_predict=v

            predicted_images.append(vector2image[y_predict])
            
        medR, recall, img_id, recipe_id = rank(img, instrs, ingrs)
    return predicted_images, img_id, recipe_id, medR, recall

## running the train method and the output losses

In [5]:

batch_size = 6

params = {'batch_size': batch_size, 'shuffle':True, 'pin_memory':True}

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
            transforms.Scale(256), # rescale the image keeping the original aspect ratio
            transforms.CenterCrop(256), # we get only the center of that rescaled
            transforms.RandomCrop(224), # random crop within the center crop 
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
            ])


training_set = Loader(data_path='data', partition = 'train', img_path='data' , transform=transform)
test_set = Loader(data_path='data', partition = 'test', img_path='data' , transform=transform)
training_generator = data.DataLoader(training_set, **params)
test_generator=data.DataLoader(test_set, **params)
 

hidden_size = 1024
lstm = LSTM(hidden_size)
resnet = RESNET()
gcn = RESNET()

train(training_generator,gcn,lstm,resnet)
#print ("Train finished!")



C:\Anaconda\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


data\train\9\8\6\5\9865f3c216.jpg
data\train\5\9\2\a\592ae682ff.jpg
data\train\c\3\9\3\c393980778.jpg
tensor([9.1834e-41])
tensor([4.0638e-44])
tensor([-137439019008.])
tensor([-267362222080.])
tensor([19.7502])
tensor([9.1834e-41])
loss 220.4025634765625
data\train\6\3\8\5\638519fbe3.jpg
tensor([2.9427e-44])
tensor([8.2677e-44])
tensor([8.2500])
tensor([2.6888e-17])
tensor([1.6255e-43])
tensor([-267361763328.])
loss 135.14261474609376
data\train\9\e\2\1\9e21803a96.jpg
data\train\a\f\c\a\afca94b507.jpg
data\train\8\c\5\1\8c516e2af5.jpg
tensor([3.6434e-44])
tensor([1.4854e-43])
tensor([4.4842e-44])
tensor([2.6625e-44])
tensor([6.1657e-44])
tensor([7.8473e-44])
loss 156.5646484375
data\train\d\f\9\f\df9f7c586b.jpg
data\train\2\3\2\9\2329cc5c00.jpg
tensor([7.0065e-45])
tensor([4.3440e-44])
tensor([7.1466e-44])
tensor([1.2892e-43])
tensor([1.3873e-43])
tensor([2.3822e-43])
loss 119.214453125
data\train\e\6\1\8\e618320906.jpg
data\train\2\0\4\a\204ae70ea5.jpg
tensor([2.9427e-44])
tensor([9.

## example of  retrieved images  for one test  recipe

In [6]:
predicted_images, img_ids, top_recipe_ids, medR, recall=test(training_generator,test_generator,lstm,resnet, gcn)
print(predicted_images[10])


data\train\4\3\7\5\4375b3aa48.jpg
data\test\2\2\1\c\221c920d7e.jpg
data\test\3\8\1\7\3817236b6e.jpg
data\test\2\4\1\e\241e22d085.jpg
data\test\0\9\7\8\0978eea9c9.jpg
data\test\0\5\a\4\05a40ab9dc.jpg
data\test\5\b\2\f\5b2fbf0ed7.jpg
data\test\2\3\4\9\23498c3a46.jpg
data\test\c\0\1\a\c01a505708.jpg
data\test\f\6\0\9\f609d06812.jpg
data\test\6\9\7\7\6977c9a0ea.jpg
data\test\7\f\7\a\7f7ab00d7f.jpg
data\test\d\d\d\c\dddc7f27a5.jpg


In [7]:
print(top_recipe_ids)
layer1 = json.load(open('../data/recipe1M_layers/layer1.json','r'))
for i in top_recipe_ids[10]:
    print(layer1[i])

{'ingredients': [{'text': '6 ounces penne'}, {'text': '2 cups Beechers Flagship Cheese Sauce (recipe follows)'}, {'text': '1 ounce Cheddar, grated (1/4 cup)'}, {'text': '1 ounce Gruyere cheese, grated (1/4 cup)'}, {'text': '1/4 to 1/2 teaspoon chipotle chili powder (see Note)'}, {'text': '1/4 cup (1/2 stick) unsalted butter'}, {'text': '1/3 cup all-purpose flour'}, {'text': '3 cups milk'}, {'text': '14 ounces semihard cheese (page 23), grated (about 3 1/2 cups)'}, {'text': '2 ounces semisoft cheese (page 23), grated (1/2 cup)'}, {'text': '1/2 teaspoon kosher salt'}, {'text': '1/4 to 1/2 teaspoon chipotle chili powder'}, {'text': '1/8 teaspoon garlic powder'}, {'text': '(makes about 4 cups)'}], 'url': 'http://www.epicurious.com/recipes/food/views/-world-s-best-mac-and-cheese-387747', 'partition': 'train', 'title': 'Worlds Best Mac and Cheese', 'id': '000018c8a5', 'instructions': [{'text': 'Preheat the oven to 350 F. Butter or oil an 8-inch baking dish.'}, {'text': 'Cook the penne 2 minu

## The vector of predicted images

In [7]:
print("Test finished!",len(predicted_images),(predicted_images[0]))

Test finished! 6 tensor([[[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],

         [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400

## the Resulted MedR and Recall Metrics

In [9]:
print("medR = {}, recall = {}".format(medR, recall))

medR = 20.7, recall = 0.09
